This test is based upon `oarphpy_test/test_spark.py#test_spark_with_custom_library()` with small modifications to run in a Jupyter notebook.  You can run this notebook by itself using:
```
jupyter-nbconvert \
            --ExecutePreprocessor.timeout=3600 \
              --to notebook --execute --output /tmp/out \
                test_spark_ships_custom_library_in_notebook.ipynb
```

The notebook assumes that `oarphpy` and Spark are installed in the system / Jupyter kernel environment.  You can do that yourself using 
```
pip install oarphpy[spark]
```
(You may need to also install Java).

First, use the notebook to create a local python module we'd like to ship with our spark jobs.

In [1]:
!mkdir -p /tmp/test_spark_ships_custom_library_in_notebook
import os; os.chdir('/tmp/test_spark_ships_custom_library_in_notebook')
!mkdir -p my_src_root/mymodule
!touch my_src_root/mymodule/__init__.py
!echo "pi = 3.14" > my_src_root/mymodule/foo.py

Test the local module

In [2]:
!cd my_src_root && $(which python || which python3) -c 'from mymodule.foo import pi; print(pi)'

3.14


Set up and start the spark session

In [3]:
import os
from oarphpy.spark import NBSpark
NBSpark.SRC_ROOT = os.path.join(os.getcwd(), 'my_src_root', 'mymodule')
spark = NBSpark.getOrCreate()

2020-01-16 23:53:31,315	oarph 6706 : Using source root /opt/avsegs-kitti-ex/my_src_root 
2020-01-16 23:53:31,343	oarph 6706 : Generating egg to /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e ...


running clean
running bdist_egg
running egg_info
writing my_src_root.egg-info/PKG-INFO
writing dependency_links to my_src_root.egg-info/dependency_links.txt
writing top-level names to my_src_root.egg-info/top_level.txt
reading manifest file 'my_src_root.egg-info/SOURCES.txt'
writing manifest file 'my_src_root.egg-info/SOURCES.txt'
installing library code to /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir
running install_lib
running build_py
copying /opt/avsegs-kitti-ex/my_src_root/mymodule/foo.py -> build/lib/mymodule
copying /opt/avsegs-kitti-ex/my_src_root/mymodule/__init__.py -> build/lib/mymodule


creating /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir
creating /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/mymodule
copying build/lib/mymodule/imagewithanno.py -> /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/mymodule
copying build/lib/mymodule/foo.py -> /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/mymodule
copying build/lib/mymodule/__init__.py -> /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/mymodule


creating /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/EGG-INFO
copying my_src_root.egg-info/PKG-INFO -> /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/EGG-INFO
copying my_src_root.egg-info/SOURCES.txt -> /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/EGG-INFO
copying my_src_root.egg-info/dependency_links.txt -> /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/EGG-INFO
copying my_src_root.egg-info/top_level.txt -> /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir/EGG-INFO


zip_safe flag not set; analyzing archive contents...


creating '/tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/my_src_root-0.0.0-py3.6.egg' and adding '/tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir' to it
removing '/tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/workdir' (and everything under it)


2020-01-16 23:53:31,406	oarph 6706 : ... done.  Egg at /tmp/op_spark_eggs_f2efe893-664a-4333-a8aa-65e4230b859e/my_src_root-0.0.0-py3.6.egg


Now test that the module gets shipped with the job in an egg

In [4]:
def test_my_lib():
    # Make sure `mymodule` is not on the default local PYTHONPATH
    import os
    os.chdir('/tmp')
    
    import re
    import mymodule

    # The module should come from the included egg
    imp_path = mymodule.__file__
    assert re.match(
      r'^(.*)spark-(.*)/mymodule-0\.0\.0-py(.+)\.egg/mymodule/__init__\.py$',
      imp_path)

    # Now verify the module itself
    from mymodule import foo
    assert foo.pi == 3.14
    
    return True
  
# Now test that the lib gets egg-ified and shipped as a SparkFile
from oarphpy import spark as S
with NBSpark.sess() as spark:
    res = S.for_each_executor(spark, lambda: test_my_lib())
    assert res and all(res)